In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings

from sklearn.metrics import mean_squared_error , r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor , AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression , Ridge , Lasso
from sklearn.metrics import r2_score , mean_absolute_error , mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor


In [2]:
df = pd.read_csv('data/raw.csv')

In [3]:
df.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [4]:
X = df.drop(columns=['math_score'] , axis = 1)

In [5]:
print(df['gender'].unique())
print(df['race_ethnicity'].unique())
print(df['parental_level_of_education'].unique())
print(df['lunch'].unique())
print(df['test_preparation_course'].unique())

['female' 'male']
['group B' 'group C' 'group A' 'group D' 'group E']
["bachelor's degree" 'some college' "master's degree" "associate's degree"
 'high school' 'some high school']
['standard' 'free/reduced']
['none' 'completed']


In [6]:
y = df['math_score']

In [10]:
numerical_features = [i for i in X.columns if X[i].dtype != "O"]
categorical_features = [j for j in X.columns if X[j].dtype == "O"]


In [12]:
from sklearn.preprocessing import OneHotEncoder , StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder" , oh_transformer , categorical_features),
        ("StandardScaler" , numeric_transformer , numerical_features)
    ]
)


In [13]:
X = preprocessor.fit_transform(X)

In [15]:
X.shape

(1000, 19)

In [18]:
from sklearn.model_selection import train_test_split
X_train , X_test , y_train , y_test = train_test_split(X,y,test_size=0.25 , random_state=19)
X_train.shape , X_test.shape , y_train.shape , y_test.shape

((750, 19), (250, 19), (750,), (250,))

In [40]:
def evaluate_model(t , p):
    mae = mean_absolute_error(t,p)
    mse = mean_squared_error(t,p)
    rmse = np.sqrt(mean_squared_error(t,p))
    r2 = r2_score(t,p)
    return mae , rmse , r2

In [41]:
models = {
    "LinearRegression" : LinearRegression(),
    "Lasso" : Lasso(),
    "Ridge" : Ridge(),
    "K Neighbors Regressor" : KNeighborsRegressor(),
    "DecisionTreeRegressor" : DecisionTreeRegressor(),
    "RandomForestRegressor" : RandomForestRegressor(),
    "XGB" : XGBRegressor(),
    "CatBoostRegressor" : CatBoostRegressor(verbose=False),
    "Adboost" : AdaBoostRegressor()
}

model_list = []
r2_list = []

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train , y_train)

    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    model_train_mae , model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)

    model_test_mae , model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))

    print('----------------------------------')
    
    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2))

    r2_list.append(model_test_r2)

    print('='*35)
    print('\n')




    





LinearRegression
Model performance for Training set
- Root Mean Squared Error: 5.3934
- Mean Absolute Error: 4.2888
- R2 Score: 0.8664
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 5.3145
- Mean Absolute Error: 4.2575
- R2 Score: 0.8931


Lasso
Model performance for Training set
- Root Mean Squared Error: 6.5763
- Mean Absolute Error: 5.2417
- R2 Score: 0.8013
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 6.7820
- Mean Absolute Error: 5.2414
- R2 Score: 0.8259


Ridge
Model performance for Training set
- Root Mean Squared Error: 5.3735
- Mean Absolute Error: 4.2617
- R2 Score: 0.8674
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 5.2401
- Mean Absolute Error: 4.2174
- R2 Score: 0.8961


K Neighbors Regressor
Model performance for Training set
- Root Mean Squared Error: 5.7560
- Mean Absolute Error: 4.6219
- R2 Score: 0.8478
------------------------

c:\Users\bhavy\OneDrive\Desktop\Machine Learning Projects\venv\Lib\site-packages\xgboost\data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):


CatBoostRegressor
Model performance for Training set
- Root Mean Squared Error: 3.0154
- Mean Absolute Error: 2.3365
- R2 Score: 0.9582
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 5.9384
- Mean Absolute Error: 4.7461
- R2 Score: 0.8665


Adboost
Model performance for Training set
- Root Mean Squared Error: 5.9750
- Mean Absolute Error: 4.8585
- R2 Score: 0.8360
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 6.3258
- Mean Absolute Error: 4.8891
- R2 Score: 0.8486




In [44]:
pd.DataFrame(list(zip(model_list , r2_list)), columns=['Model' , 'r2_score']).sort_values(by=["r2_score"] , ascending=False)

,Model,r2_score
2,Ridge,0.896085
0,LinearRegression,0.893113
7,CatBoostRegressor,0.866545
5,RandomForestRegressor,0.850316
6,XGB,0.850217
8,Adboost,0.848563
1,Lasso,0.825933
3,K Neighbors Regressor,0.795467
4,DecisionTreeRegressor,0.681848
